## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
# Import API key
from config import g_key

# Configure gmaps API key
#gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = os.path.join("C:\\Users\\Nichelle\\Desktop\\Classwork\\World_Weather_Analysis\\Weather_Database\\WeatherPy_Database.csv")
city_data_df = pd.read_csv("C:\\Users\\Nichelle\\Desktop\\Classwork\\World_Weather_Analysis\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head

<bound method NDFrame.head of      City_ID               City Country      Lat       Lng  Max Temp  \
0          0            mataura      NZ -46.1927  168.8643     54.48   
1          1             albany      US  42.6001  -73.9662     66.92   
2          2              thoen      TH  17.6129   99.2161     86.74   
3          3             hobart      AU -42.8794  147.3294     57.25   
4          4         butaritari      KI   3.0707  172.7902     82.45   
..       ...                ...     ...      ...       ...       ...   
681      681       vangaindrano      MG -23.3500   47.6000     66.11   
682      682           impfondo      CG   1.6381   18.0667     69.80   
683      683                sur      OM  22.5667   59.5289     86.76   
684      684          geraldton      AU -28.7667  114.6000     59.41   
685      685  krasnoye-na-volge      RU  57.5125   41.2419     63.03   

     Humidity  Cloudiness  Wind Speed Current Description  Rain (inches)  \
0          64          28    

In [28]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [29]:
gmaps.configure(api_key=g_key)

In [30]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [31]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 80


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,mataura,NZ,-46.1927,168.8643,54.48,64,28,11.86,scattered clouds,0.00,0
1,1,albany,US,42.6001,-73.9662,66.92,70,85,3.44,overcast clouds,0.00,0
3,3,hobart,AU,-42.8794,147.3294,57.25,69,20,9.22,few clouds,0.00,0
5,5,mandera,KE,3.9366,41.8670,73.20,67,43,14.83,scattered clouds,0.00,0
8,8,srednekolymsk,RU,67.4500,153.6833,79.56,33,100,13.20,overcast clouds,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
680,680,iguguno,TZ,-4.5697,34.6275,56.07,77,6,10.49,clear sky,0.00,0
681,681,vangaindrano,MG,-23.3500,47.6000,66.11,90,96,2.59,light rain,0.17,0
682,682,impfondo,CG,1.6381,18.0667,69.80,90,96,2.93,overcast clouds,0.00,0
684,684,geraldton,AU,-28.7667,114.6000,59.41,51,100,14.97,overcast clouds,0.00,0


In [5]:
# 4a. Determine if there are any empty rows.


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,mataura,NZ,54.48,-46.1927,168.8643,
1,albany,US,66.92,42.6001,-73.9662,
3,hobart,AU,57.25,-42.8794,147.3294,
5,mandera,KE,73.20,3.9366,41.8670,
8,srednekolymsk,RU,79.56,67.4500,153.6833,
10,eregli,TR,60.80,37.5133,34.0467,
11,busselton,AU,56.62,-33.6500,115.3333,
12,rikitea,PF,75.16,-23.1203,-134.9692,
13,chuy,UY,55.38,-33.6971,-53.4616,
15,suez,EG,77.94,29.9737,32.5263,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,mataura,NZ,54.48,-46.1927,168.8643,Ellie's Villa
1,albany,US,66.92,42.6001,-73.9662,
3,hobart,AU,57.25,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
5,mandera,KE,73.20,3.9366,41.8670,Red Sea Resort & Suites Mandera County
8,srednekolymsk,RU,79.56,67.4500,153.6833,
10,eregli,TR,60.80,37.5133,34.0467,Hotel Fuat Bey
11,busselton,AU,56.62,-33.6500,115.3333,Observatory Guest House
12,rikitea,PF,75.16,-23.1203,-134.9692,People ThankYou
13,chuy,UY,55.38,-33.6971,-53.4616,Nuevo Hotel Plaza
15,suez,EG,77.94,29.9737,32.5263,Green House


In [45]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City          517
Country       513
Max Temp      517
Lat           517
Lng           517
Hotel Name    517
dtype: int64

In [46]:
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,mataura,NZ,54.48,-46.1927,168.8643,Ellie's Villa
1,albany,US,66.92,42.6001,-73.9662,
3,hobart,AU,57.25,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
5,mandera,KE,73.20,3.9366,41.8670,Red Sea Resort & Suites Mandera County
8,srednekolymsk,RU,79.56,67.4500,153.6833,
...,...,...,...,...,...,...
680,iguguno,TZ,56.07,-4.5697,34.6275,
681,vangaindrano,MG,66.11,-23.3500,47.6000,Hotel Fanilo
682,impfondo,CG,69.80,1.6381,18.0667,Auberge La Place de Jules
684,geraldton,AU,59.41,-28.7667,114.6000,Broadwater Mariner Resort


In [53]:
# 8a. Create the output File (CSV)
#output_data_file
# 8b. Export the City_Data into a csv
#hotel_df.to_csv

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations)
# 11b. Display the figure


In [60]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))